# 最小费用最大流

# 【模板】最小费用最大流

## 题目描述

给出一个包含 $n$ 个点和 $m$ 条边的有向图（下面称其为网络） $G=(V,E)$，该网络上所有点分别编号为 $1 \sim n$，所有边分别编号为 $1\sim m$，其中该网络的源点为 $s$，汇点为 $t$，网络上的每条边 $(u,v)$ 都有一个流量限制 $w(u,v)$ 和单位流量的费用 $c(u,v)$。

你需要给每条边 $(u,v)$ 确定一个流量 $f(u,v)$，要求：

1.  $0 \leq f(u,v) \leq w(u,v)$（每条边的流量不超过其流量限制）；
2. $\forall p \in \{V \setminus \{s,t\}\}$，$\sum_{(i,p) \in E}f(i,p)=\sum_{(p,i)\in E}f(p,i)$（除了源点和汇点外，其他各点流入的流量和流出的流量相等）；
3. $\sum_{(s,i)\in E}f(s,i)=\sum_{(i,t)\in E}f(i,t)$（源点流出的流量等于汇点流入的流量）。

定义网络 $G$ 的流量 $F(G)=\sum_{(s,i)\in E}f(s,i)$，网络 $G$ 的费用 $C(G)=\sum_{(i,j)\in E} f(i,j) \times c(i,j)$。

你需要求出该网络的**最小费用最大流**，即在 $F(G)$ 最大的前提下，使 $C(G)$ 最小。

## 输入格式

输入第一行包含四个整数 $n,m,s,t$，分别代表该网络的点数 $n$，网络的边数 $m$，源点编号 $s$，汇点编号 $t$。

接下来 $m$ 行，每行四个整数 $u_i,v_i,w_i,c_i$，分别代表第 $i$ 条边的起点，终点，流量限制，单位流量费用。

## 输出格式

输出两个整数，分别为该网络的最大流 $F(G)$，以及在 $F(G)$ 最大的前提下，该网络的最小费用 $C(G)$。

## 样例 #1

### 样例输入 #1

```
4 5 4 3
4 2 30 2
4 3 20 3
2 3 20 1
2 1 30 9
1 3 40 5
```

### 样例输出 #1

```
50 280
```

## 提示

对于 $100\%$ 的数据，$1 \leq n \leq 5\times 10^3$，$1 \leq m \leq 5 \times 10^4$，$1 \leq s,t \leq n$，$u_i \neq v_i$，$0 \leq w_i,c_i \leq 10^3$，且该网络的最大流和最小费用 $\leq 2^{31}-1$。

输入数据随机生成。

In [18]:
from collections import deque
from typing import List
import numpy as np

class edge:
    def __init__(self,to:int,w:int,nxt:int,dis:int):
        self.to=to
        self.w=w#w指的是边的容量
        self.nxt=nxt
        self.dis=dis#dis指的是边的长度（即费用）
class mcmf:
    def __init__(self,n:int,m:int,s:int,t:int,edges:List[List[int]]):
        self.n=n
        self.m=m
        self.s=s
        self.t=t
        self.cnt=0
        self.maxflow=0
        self.mincost=0
        self.e=[edge(0,0,0,0)]#这条边只是占个位
        self.head=np.zeros(n+5,dtype=int)
        #self.dep=np.zeros(n+5)
        self.dis=np.ones(self.n+5)*np.inf#dis表示最短路长度（的估计值）
        self.incf=np.ones(self.n+5)*np.inf#incf表示从当前点流出的最大流量
        self.pre=np.zeros(self.n+5,dtype=int)#pre表示前驱
        for i in edges:#链式前向星存图
            u,v,w,x=i[0],i[1],i[2],i[3]
            self.add(u,v,w,x)
            self.add(v,u,0,-x)#反向边的费用取相反数
    def solve(self):
#         while self.bfs()!=0:#当汇点的深度为初始值（0），说明已经没有增广路径
#             self.ans+=self.dfs(self.s,1e18)
#         return self.ans
        while self.spfa():#当存在增广路时
            self.maxflow+=self.incf[self.t]#加上增广路的流量
            self.mincost+=self.dis[self.t]*self.incf[self.t]#加上费用
            x=self.t
            while x!=self.s:#从汇点开始沿着增广路往回遍历
                i=self.pre[x]
                self.e[i].w-=self.incf[self.t]
                if i%2==1:
                    self.e[i+1].w+=self.incf[self.t]
                    x=self.e[i+1].to
                else:
                    self.e[i-1].w+=self.incf[self.t]
                    x=self.e[i-1].to
        return (self.maxflow,self.mincost)
                
            
        
    def add(self,u:int,v:int,w:int,x:int):
        self.cnt+=1
        self.e.append(edge(to=v,w=w,nxt=self.head[u],dis=x))
        self.head[u]=self.cnt
    def spfa(self)->bool:#根据spfa找到长度最小的一条增广路径,返回是否存在增广
        q=deque([])
        q.append(self.s)
        vis=np.zeros(self.n+5)#vis表示节点是否存在队列中
        self.dis=np.ones(self.n+5)*np.inf#dis表示最短路长度（的估计值）
        self.incf=np.ones(self.n+5)*np.inf#incf表示从当前点流出的最大流量
        self.dis[self.s]=0
        while q:
            tmp=q[0]
            i=self.head[tmp]
            while i:
                if self.e[i].w==0:#只考虑有剩余容量的边
                    i=self.e[i].nxt
                    continue
                tt=self.e[i].to
                if self.dis[tt]>self.dis[tmp]+self.e[i].dis:
                    self.dis[tt]=self.dis[tmp]+self.e[i].dis#更新长度
                    self.incf[tt]=min(self.incf[tmp],self.e[i].w)#更新流量
                    self.pre[tt]=i
                    if vis[tt]==0:#若tt没有在队列中，将其入队
                        q.append(tt)
                i=self.e[i].nxt
            q.popleft()
            vis[tmp]=0#出队后修改标记
        if self.dis[self.t]==np.inf:
            return 0
        else:
            return 1
    #现在位于now，能够允许的最大流量为maxn,返回实际从该点流出的流量
                
# n,m,s,t=4,5,4,3
# edges=[[4,2,30,2],[4,3,20,3],[2,3,20,1],[2,1,30,9],[1,3,40,5]]
# x,y=mcmf(n,m,s,t,edges).solve()
# print(int(x),int(y))

n,m,s,t= map(int, input().split())
edges=[]
for i in range(m):
    edges.append(list(map(int, input().split())))
x,y=mcmf(n,m,s,t,edges).solve()
print(int(x),int(y))


4 5 4 3
4 2 30 2
4 3 20 3
2 3 20 1
2 1 30 9
1 3 40 5
50 280
